In [12]:
import pandas as pd
import bioframe
import json
print(bioframe.__version__)

chrmlens = bioframe.read_chromsizes('/project/fudenber_735/genomes/mm10/mm10.chrom.sizes.reduced')


0.4.1


In [9]:
base_dir = '/project/fudenber_735/tensorflow_models/akita/v2/models/'
head_i = 1 #mouse
model_num = 7
model_dir = base_dir+"f"+str(model_num)+"c0/train/"
params_file = model_dir+'params.json'
model_file  = model_dir+'/model'+str(head_i)+'_best.h5'
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params['model']
    params_train = params['train']
seq_length = params_model['seq_length']

In [10]:
# reading SMF binding site table
bs = pd.read_table('binding.sites.tsv', sep='\t')

In [11]:
bs.rename(columns={'seqnames':'chrom'},inplace=True)
bs = bs[bs.chrom.isin(['chrX','chrY']) == False]
print(bs.shape)

(10015, 12)


In [5]:
bs_filtered = []
s = 0
for chrm in chrmlens.index:
    if chrm in ['chrY','chrX','chrM']: continue
    bs_chrm = bs.loc[bs['chrom']==chrm]
    s += len(bs_chrm)
    bs_filtered.append(bs_chrm.loc[ 
        (bs_chrm['end'].values + seq_length//2 + 100) < chrmlens[chrm],:])
bs_filtered = pd.concat(bs_filtered)
bs_filtered.reset_index(inplace=True, drop=True)
print(s, bs_filtered.shape)

10015 (9991, 12)


In [6]:
bs_filtered.to_csv('binding.sites.filtered.mm10.tsv', sep='\t',index=False)